# OPERA L2-RTC Validation 

OPERA L2-RTC Validation is performed on OPERA bursts that are merged into the shape of the source Sentinel-1 scenes.

Each validation workflow is a collection of notebooks in this repository. They may be run piecemeal, on individual scenes, but it is time consuming.

**This notebook makes it easy to run validation modules in bulk, on all of the data submitted for validation.** 

1. Select a validation module
1. Select a site and orbital path combination
1. Download non-OPERA data used in validation
1. Run a bulk validation script
    1. Downloads and mosaics the bursts
    1. Runs validation workflows
    
After running validation modules from this notebook, the input and output validation data can be found in the `../OPERA_L2-RTC_CalVal` directory.

In [12]:
from getpass import getpass
import os
from pathlib import Path
import subprocess
from typing import Union
from zipfile import ZipFile 

from opensarlab_lib import work_dir

import ipywidgets as widgets
from ipywidgets import Layout

## 1. Select Validation Module

In [2]:
calval_types = [
    "Absolute Geolocation Evaluation",
    "Flattening",
    "Coregistration",
]

calval_module = widgets.RadioButtons(
    options=calval_types,
    description='',
    disabled=False,
    layout=Layout(min_width="1000px")
    )

print("Select an OPERA-RTC validation analysis to perform.")
display(calval_module)

Select an OPERA-RTC validation analysis to perform.


RadioButtons(layout=Layout(min_width='1000px'), options=('Absolute Geolocation Evaluation', 'Flattening', 'Cor…

## 2. Select a validation site and orbital path

In [3]:
if "Absolute" in calval_module.value:
    
    sites = [
        "California, Ascending Orbit, Orbital Path 64",
    ]
    script = "bulk_papermill_OPERA_RTC_absolute_geolocation_evaluation.py"
elif calval_module.value == 'Coregistration':
    sites = [
        "Delta Junction, Ascending Orbit, Path 94",
        "Delta Junction, Descending Orbit, Path 160",
        "Brazil, Descending Orbit, Path 170",
    ]
    script = "bulk_papermill_OPERA_RTC_coregistration.py"
elif calval_module.value == 'Flattening':
    sites = [
        "Vermont, Ascending Orbit, Path 135",
        "Vermont, Descending Orbit, Path 113",
        "Delta Junction, Ascending Orbit, Path 94",
        "Delta Junction, Descending Orbit, Path 160",
        "Brazil, Descending Orbit, Path 39",
    ]
    script = "bulk_papermill_OPERA_RTC_flattening.py"
    
else:
    raise Exception(f"{calval_module.value} analysis option not yet added to this notebook")
    
site_options = widgets.RadioButtons(
    options=sites,
    description='',
    disabled=False,
    layout=Layout(min_width="1000px")
    )

print("Select a site/path for the validation analysis.")
display(site_options)

Select a site/path for the validation analysis.


RadioButtons(layout=Layout(min_width='1000px'), options=('California, Ascending Orbit, Orbital Path 64',), val…

## 3. Select whether to download and validate data or validate previously downloaded data

In [4]:
download_mosaic = [
    'Download, mosaic, and validate OPERA sample burst data',
    'Validate previously downloaded and mosaicked data'
]

download_options = widgets.RadioButtons(
    options=download_mosaic,
    description='',
    disabled=False,
    layout=Layout(min_width="1000px")
    )

print("Download and mosaic OPERA sample burst data or validate previously prepared mosaics?")
display(download_options)

if calval_module.value == 'Coregistration':
    print("\nSave or delete intermediary data produced during validation?")
    delete_options = widgets.RadioButtons(
        options=['Save intermediary data', 'Delete intermediary data'],
        description='',
        disabled=False,
        layout=Layout(min_width="1000px")
        )
    display(delete_options)

Download and mosaic OPERA sample burst data or validate previously prepared mosaics?


RadioButtons(layout=Layout(min_width='1000px'), options=('Download, mosaic, and validate OPERA sample burst da…

## 4. Download [Copernicus Global Land Cover (100m)](https://lcviewer.vito.be/download) and [NISAR Calibration Corner Reflector](https://uavsar.jpl.nasa.gov/cgi-bin/calibration-nisar.pl) data.  

In [8]:
if "Download" in download_options.value:
    calval_pth = Path.cwd().parent / "OPERA_L2-RTC_CalVal"
    zip_pth = Path.cwd().parent / "OPERA_L2-RTC_CalVal.zip"

    if not zip_pth.exists() and not calval_pth.exists():
        download_cmd = f'wget --no-check-certificate --no-proxy "https://asf-opera-rtc-calval.s3.us-west-2.amazonaws.com/OPERA_L2-RTC_LC_CR_Input_Data/OPERA_L2-RTC_CalVal.zip" -q --show-progress -P {Path.cwd().parent}'
        subprocess.run(download_cmd, stdout=None, stderr=subprocess.PIPE, shell=True)
    if zip_pth.exists() and not calval_pth.exists():
        with ZipFile(zip_pth, 'r') as z:  
            z.extractall(path=Path.cwd().parent)    

## Enter your EarthData Login Bear Token

[Instructions to generate an EDL Bearer Token](https://urs.earthdata.nasa.gov/documentation/for_users/user_token)

In [15]:
EDL_token = getpass("EDL Bearer Token")

EDL Bearer Token ········


## 5. Run bulk validation script

In [46]:
download = '--skip_download' if 'Validate' in download_options.value else ''
delete = '--delete_intermediary' if (calval_module.value == 'Coregistration' and 'delete_options' in locals() and 'Delete' in delete_options.value) else ''
script_dir = Path.cwd()/"bulk_validation_scripts"
with work_dir(script_dir):
    site = site_options.value.split(',')[0]
    orbital_path = site_options.value.split(' ')[-1]
    subprocess.run([f'python {script} --site "{site}" --orbital_path {orbital_path} --EDL_token {EDL_token} {download} {delete}'], shell=True)

99


  0%|          | 0/99 [00:00<?, ?it/s]/home/jovyan/.local/envs/opera_calval_rtc/lib/python3.11/site-packages/asf_search/download/download.py:68: UserWarning: File already exists, skipping download: /home/jovyan/OPERA_L2-RTC_CalVal/OPERA_RTC_ALE_California_64/input_OPERA_data/OPERA_L2-RTC_S1A_IW_SLC__1SDV_20201107T015034_20201107T015102_035136_0419CE_F4EF_30_v1.0/vv_bursts/OPERA_L2_RTC-S1_T064-135527-IW2_20201107T015057Z_20230928T223020Z_S1A_30_v1.0_VV.tif
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/home/jovyan/.local/envs/opera_calval_rtc/lib/python3.11/site-packages/asf_search/download/download.py:68: UserWarning: File already exists, skipping download: /home/jovyan/OPERA_L2-RTC_CalVal/OPERA_RTC_ALE_California_64/input_OPERA_data/OPERA_L2-RTC_S1A_IW_SLC__1SDV_20201107T015034_20201107T015102_035136_0419CE_F4EF_30_v1.0/vv_bursts/OPERA_L2_RTC-S1_T064-135527-IW3_20201107T015058Z_20230928T223020Z_S1A_30_v1.0_VV.tif
  warnings.warn(f'File alre

0...10...20...30...40...50...60...70...80...90...100 - done.


  2%|▏         | 2/99 [00:07<05:51,  3.62s/it]

0...10...20...30...40...50...60...70...80...90...100 - done.


...

0...10...20...30...40...50...60...70...80...90...100 - done.


 86%|████████▌ | 85/99 [38:43<06:41, 28.69s/it]

Merging bursts -> /home/jovyan/OPERA_L2-RTC_CalVal/OPERA_RTC_ALE_California_64/input_OPERA_data/OPERA_L2-RTC_S1A_IW_SLC__1SDV_20220326T015033_20220326T015101_042486_051118_6B3C_30_v1.0/OPERA_L2_RTC-S1_VV_S1A_IW_SLC__1SDV_20220326T015033_20220326T015101_042486_051118_6B3C_30_v1.0_mosaic.tif
/home/jovyan/OPERA_L2-RTC_CalVal/OPERA_RTC_ALE_California_64/input_OPERA_data/OPERA_L2-RTC_S1A_IW_SLC__1SDV_20210927T015040_20210927T015108_039861_04B734_A229_30_v1.0/vv_bursts
28
Merging bursts -> /home/jovyan/OPERA_L2-RTC_CalVal/OPERA_RTC_ALE_California_64/input_OPERA_data/OPERA_L2-RTC_S1A_IW_SLC__1SDV_20210927T015040_20210927T015108_039861_04B734_A229_30_v1.0/OPERA_L2_RTC-S1_VV_S1A_IW_SLC__1SDV_20210927T015040_20210927T015108_039861_04B734_A229_30_v1.0_mosaic.tif
/home/jovyan/OPERA_L2-RTC_CalVal/OPERA_RTC_ALE_California_64/input_OPERA_data/OPERA_L2-RTC_S1A_IW_SLC__1SDV_20210810T015038_20210810T015105_039161_049F3C_B9BF_30_v1.0/vv_bursts
28
Merging bursts -> /home/jovyan/OPERA_L2-RTC_CalVal/OPERA_R

 87%|████████▋ | 86/99 [39:23<05:57, 27.48s/it]
Traceback (most recent call last):
  File "/home/jovyan/calval-RTC/bulk_validation_scripts/bulk_papermill_OPERA_RTC_absolute_geolocation_evaluation.py", line 152, in <module>
    main()
  File "/home/jovyan/calval-RTC/bulk_validation_scripts/bulk_papermill_OPERA_RTC_absolute_geolocation_evaluation.py", line 147, in main
    download_mosaic_data(parent_data_dir, args)
  File "/home/jovyan/calval-RTC/bulk_validation_scripts/bulk_papermill_OPERA_RTC_absolute_geolocation_evaluation.py", line 71, in download_mosaic_data
    asf_search.download_urls(urls=vv_urls, path=vv_burst_dir, session=asf_search.ASFSession().auth_with_token(args.EDL_token))
  File "/home/jovyan/.local/envs/opera_calval_rtc/lib/python3.11/site-packages/asf_search/download/download.py", line 41, in download_urls
    download_url(url=url, path=path, session=session)
  File "/home/jovyan/.local/envs/opera_calval_rtc/lib/python3.11/site-packages/asf_search/download/download.py"

Merging bursts -> /home/jovyan/OPERA_L2-RTC_CalVal/OPERA_RTC_ALE_California_64/input_OPERA_data/OPERA_L2-RTC_S1A_IW_SLC__1SDV_20210412T015031_20210412T015059_037411_0468C7_6259_30_v1.0/OPERA_L2_RTC-S1_VV_S1A_IW_SLC__1SDV_20210412T015031_20210412T015059_037411_0468C7_6259_30_v1.0_mosaic.tif
/home/jovyan/OPERA_L2-RTC_CalVal/OPERA_RTC_ALE_California_64/input_OPERA_data/OPERA_L2-RTC_S1A_IW_SLC__1SDV_20220407T015033_20220407T015101_042661_051705_FD8D_30_v1.0/vv_bursts
28


In [31]:
import asf_search
url = "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135528-IW1_20210412T015056Z_20230928T222655Z_S1A_30_v1.0_VV.tif"
asf_search.download_urls(urls=[url], path=Path.cwd(), session=asf_search.ASFSession().auth_with_token(EDL_token))

In [38]:
nine8 = ['https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135518-IW1_20220712T015039Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135518-IW2_20220712T015040Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135518-IW3_20220712T015041Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135519-IW1_20220712T015042Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135519-IW2_20220712T015043Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135519-IW3_20220712T015044Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135520-IW1_20220712T015045Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135520-IW2_20220712T015046Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135520-IW3_20220712T015047Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135521-IW1_20220712T015048Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135521-IW2_20220712T015048Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135521-IW3_20220712T015049Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135522-IW1_20220712T015050Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135522-IW2_20220712T015051Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135522-IW3_20220712T015052Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135523-IW1_20220712T015053Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135523-IW2_20220712T015054Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135523-IW3_20220712T015055Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135524-IW1_20220712T015056Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135524-IW2_20220712T015057Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135524-IW3_20220712T015058Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135525-IW1_20220712T015059Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135525-IW2_20220712T015059Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135525-IW3_20220712T015100Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135526-IW1_20220712T015101Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135526-IW2_20220712T015102Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135526-IW3_20220712T015103Z_20230928T214154Z_S1A_30_v1.0_VV.tif',
 'https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135527-IW1_20220712T015104Z_20230928T214154Z_S1A_30_v1.0_VV.tif']

In [39]:
asf_search.download_urls(urls=nine8, path=Path.cwd()/"98", session=asf_search.ASFSession().auth_with_token(EDL_token))

In [48]:
atesix = [
    "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135522-IW2_20210412T015040Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135523-IW2_20210412T015043Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135524-IW1_20210412T015045Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135522-IW1_20210412T015039Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135523-IW1_20210412T015042Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135520-IW1_20210412T015034Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135521-IW1_20210412T015036Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135525-IW2_20210412T015048Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135527-IW1_20210412T015053Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135520-IW2_20210412T015035Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135522-IW3_20210412T015041Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135524-IW2_20210412T015046Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135521-IW2_20210412T015037Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135527-IW3_20210412T015055Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135524-IW3_20210412T015047Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135526-IW2_20210412T015051Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135526-IW3_20210412T015052Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135519-IW1_20210412T015031Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135520-IW3_20210412T015036Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135525-IW3_20210412T015049Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135526-IW1_20210412T015050Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135527-IW2_20210412T015054Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135519-IW2_20210412T015032Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135519-IW3_20210412T015033Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135521-IW3_20210412T015038Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135523-IW3_20210412T015044Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135525-IW1_20210412T015048Z_20230928T222655Z_S1A_30_v1.0_VV.tif", "https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T064-135528-IW1_20210412T015056Z_20230928T222655Z_S1A_30_v1.0_VV.tif",
]

In [49]:
asf_search.download_urls(urls=atesix, path=Path.cwd()/"86", session=asf_search.ASFSession().auth_with_token(EDL_token))